# FiftyOne WandB Plugin Demo

A minimal guide to using the FiftyOne WandB plugin operators.

**Setup**: Set your WandB credentials first:
```bash
export FIFTYONE_WANDB_API_KEY="your-api-key"
export FIFTYONE_WANDB_ENTITY="your-username"
export FIFTYONE_WANDB_PROJECT="demo-project"
```


In [ ]:
import fiftyone as fo
import fiftyone.operators as foo
import fiftyone.zoo as foz
import wandb

# Load sample dataset
dataset = foz.load_zoo_dataset("quickstart")
print(f"Loaded {len(dataset)} samples")


---

## 1. Log FiftyOne View to WandB

**Purpose**: Track your training/validation data with full label support.

**When**: Before training, to version your datasets.

**What it logs**:
- Sample IDs for reproducibility
- All labels (with per-class counts)
- View serialization
- Optional: embeddings

**Expected artifact**: Dataset artifact with WandB Table containing labels.


In [ ]:
# Create a training view
train_view = dataset.match_tags("validation").take(20)

# Initialize WandB run
run = wandb.init(project="demo-project", name="training-run-1")
run_id = run.id
wandb.finish()

# Log view to WandB
operator = foo.get_operator("@harpreetsahota/wandb/log_fiftyone_view_to_wandb")
result = operator(
    train_view,
    project="demo-project",
    run_id=run_id,
    include_labels=True  # Logs all label types with counts
)

print(f"✓ Logged {result['samples_logged']} samples")
print(f"Artifact: {result['artifact_name']}")
print(f"WandB: {result['wandb_url']}")


---

## 2. Log Model Predictions to WandB

**Purpose**: Track model inference outputs with detection-level granularity.

**When**: After running `apply_model()`, to track model performance.

**What it logs**:
- Label IDs (for selecting specific predictions)
- Prediction counts and confidence stats
- Class distribution
- Model configuration
- Optional: per-sample prompts (for VLMs)

**Expected artifact**: Predictions artifact with lightweight table (no images by default).


In [ ]:
# Apply model (if predictions don't exist)
if "predictions" not in dataset.get_field_schema():
    model = foz.load_zoo_model("yolov8n-coco-torch")
    dataset.apply_model(model, label_field="predictions")

# Log predictions
operator = foo.get_operator("@harpreetsahota/wandb/log_model_predictions")
result = operator(
    dataset.take(20),
    model_name="yolov8n",
    predictions_field="predictions",
    project="demo-project"
)

print(f"✓ Logged {result['total_predictions']} predictions")
print(f"Classes: {result['num_classes']}")
print(f"WandB: {result['wandb_url']}")


### VLM Example (with prompts)

Track Vision Language Model outputs with per-sample prompts and global configuration:


In [ ]:
# Add sample-specific prompts
for sample in dataset.take(5):
    sample["vlm_prompt"] = "Describe this image in detail"
    sample["vlm_output"] = "Example VLM caption..."  # Your VLM output here
    sample.save()

# Log with prompts and config
result = operator(
    dataset.take(5),
    model_name="gpt4-vision",
    predictions_field="vlm_output",
    project="demo-project",
    model_config={
        "system_prompt": "You are an expert analyst.",
        "temperature": 0.7,
        "max_tokens": 150
    },
    prompt_field="vlm_prompt"  # Sample-specific prompts
)

print(f"✓ Logged VLM predictions with prompts")


---

## 3. Complete Workflow: Dataset → Model → Predictions

Track the full ML lifecycle:


In [ ]:
# 1. Create and log training view
train_view = dataset.match_tags("validation").take(30)

run = wandb.init(project="workflow-demo", name="experiment-1")
run_id = run.id
wandb.finish()

log_view = foo.get_operator("@harpreetsahota/wandb/log_fiftyone_view_to_wandb")
view_result = log_view(train_view, project="workflow-demo", run_id=run_id, include_labels=True)
print(f"✓ Training data: {view_result['artifact_name']}")

# 2. Apply model
if "predictions" not in dataset.get_field_schema():
    model = foz.load_zoo_model("yolov8n-coco-torch")
    dataset.apply_model(model, label_field="predictions")
print(f"✓ Model applied")

# 3. Log predictions
log_preds = foo.get_operator("@harpreetsahota/wandb/log_model_predictions")
pred_result = log_preds(
    train_view,
    model_name="yolov8n",
    predictions_field="predictions",
    project="workflow-demo"
)
print(f"✓ Predictions: {pred_result['artifact_name']} ({pred_result['total_predictions']} predictions)")
print(f"\\n🎉 Complete! View: https://wandb.ai/[entity]/workflow-demo")


---

## 4. Download and Use Artifacts

Access logged data from WandB:


In [ ]:
api = wandb.Api()

# Get dataset artifact
artifact_name = view_result['artifact_name']
dataset_artifact = api.artifact(f"voxel51/workflow-demo/{artifact_name}:latest")

# Recreate view using sample IDs
sample_ids = dataset_artifact.metadata['sample_ids']
recreated_view = dataset.select(sample_ids)
print(f"✓ Recreated training view: {len(recreated_view)} samples")

# Get predictions artifact
pred_artifact_name = pred_result['artifact_name']
preds_artifact = api.artifact(f"voxel51/workflow-demo/{pred_artifact_name}:latest")

# Get low-confidence detections
low_conf_ids = preds_artifact.metadata.get('low_confidence_label_ids', [])
if low_conf_ids:
    uncertain_view = dataset.select_labels(ids=low_conf_ids[:5])
    print(f"✓ Selected {uncertain_view.count('predictions.detections')} uncertain detections")


---

## 5. Using in FiftyOne App

Interactive usage with target view selection:


In [ ]:
# Launch FiftyOne App
session = fo.launch_app(dataset)

# In the App UI, you can:
# 1. Select samples manually (click in grid)
# 2. Press backtick key to open operators menu
# 3. Search for WandB operators
# 4. Select target: Dataset, Current view, or Selected samples
# 5. Execute with your parameters

print("✓ FiftyOne App launched - use operators menu in the App")


---

## Summary

**Key Operators:**

1. **`log_fiftyone_view_to_wandb`** - Track datasets/views with labels
2. **`log_model_predictions`** - Track model outputs with label IDs

**Features:**
- ✅ All 15 FiftyOne label types supported
- ✅ Per-class counts: "person(5), car(3)"
- ✅ Sample IDs and Label IDs for reproducibility
- ✅ VLM support with prompts and model config
- ✅ Target view selection (Dataset, View, Selected samples)
- ✅ Lightweight by default (no images)

**Documentation**: See `wandb_plugin_info/` directory for complete guides.
